In [ ]:
!pip install bert-serving-server
!pip install bert-serving-client

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=1f7a00792d92d53e52c6d56233bdb3f54338b7929bedb7d339a91781e2bb0482
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [1]:
import pandas as pd
import numpy
import re
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords

In [3]:
from gensim.models import Word2Vec
import gensim.downloader as api

In [5]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=ee4a590cf31b5a1b23a6fad4481514ce179e54b15b10e8b1db4cccfc5e96aed5
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [7]:
import wikipedia

In [8]:
df = pd.read_csv("WikiQA-dev copy.txt", sep="\t")


In [9]:
df.head(10)

,Question ?,Answer,Unnamed: 2,0
0,How are epithelial tissues joined together ?,Cross section of sclerenchyma fibers in plant ...,0.0,NaN
1,How are epithelial tissues joined together ?,Microscopic view of a histologic specimen of h...,0.0,NaN
2,How are epithelial tissues joined together ?,"In Biology , Tissue is a cellular organization...",0.0,NaN
3,How are epithelial tissues joined together ?,A tissue is an ensemble of similar cells from ...,0.0,NaN
4,How are epithelial tissues joined together ?,Organs are then formed by the functional group...,0.0,NaN
5,How are epithelial tissues joined together ?,"The study of tissue is known as histology or ,...",0.0,NaN
6,How are epithelial tissues joined together ?,The classical tools for studying tissues are t...,0.0,NaN
7,How are epithelial tissues joined together ?,"In the last couple of decades , developments i...",0.0,NaN
8,How are epithelial tissues joined together ?,"With these tools , the classical appearances o...",0.0,NaN
9,"how big is bmc software in houston , tx","BMC Software , Inc. is an American company spe...",0.0,NaN


In [ ]:
df.columns

Index(['Question ?', 'Answer', 'Unnamed: 2', '0'], dtype='object')

In [ ]:
df.drop('0',axis=1)
df.drop('Unnamed: 2',axis=1)

,Question ?,Answer,0
0,How are epithelial tissues joined together ?,Cross section of sclerenchyma fibers in plant ...,NaN
1,How are epithelial tissues joined together ?,Microscopic view of a histologic specimen of h...,NaN
2,How are epithelial tissues joined together ?,"In Biology , Tissue is a cellular organization...",NaN
3,How are epithelial tissues joined together ?,A tissue is an ensemble of similar cells from ...,NaN
4,How are epithelial tissues joined together ?,Organs are then formed by the functional group...,NaN
...,...,...,...
35418,what is section eight housing,A tenant who leaves a subsidized project will ...,NaN
35419,what is section eight housing,The United States Department of Housing and Ur...,NaN
35420,what is the main type of restaurant,Restaurants categorized by type and informatio...,NaN
35421,what is us dollar worth based on,U.S. Federal Reserve notes in the mid-1990s,NaN


In [10]:
df.columns.isna().sum()

0

In [11]:
def clean_sentence(sentence,stopwords =False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]','',sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)

  return sentence

In [12]:
def get_cleaned_sentences(df , stopwords=False):
  sents = df[["Question ?"]];# replace with questions
  cleaned_sentences = []
  for index,row in df.iterrows():
    cleaned = clean_sentence(row["Question ?"],stopwords);# replace with questions
    cleaned_sentences.append(cleaned);
  return cleaned_sentences;
cleaned_sentences = get_cleaned_sentences(df,stopwords=True)
print("cleaned sentences without stop words");
print(cleaned_sentences);
print("\n")
cleaned_sentences_with_stopwords = get_cleaned_sentences(df,stopwords=False)
print("cleaned sentences with stopwords");
print(cleaned_sentences_with_stopwords);

cleaned sentences without stop words
['epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'epithelial tissues joined', 'big bmc software houston tx', 'big bmc software houston tx', 'big bmc software houston tx', 'big bmc software houston tx', 'big bmc software houston tx', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'long love lucy air', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando christian perez famous', 'armando

In [13]:
sentences = cleaned_sentences_with_stopwords
sentence_words = [[word for word in document.split()]
                  for document in sentences]

In [14]:
dictionary = corpora.Dictionary(sentence_words)

for key,value in dictionary.items():
  print(key,':',value)

0 : are
1 : epithelial
2 : how
3 : joined
4 : tissues
5 : together
6 : big
7 : bmc
8 : houston
9 : in
10 : is
11 : software
12 : tx
13 : air
14 : i
15 : long
16 : love
17 : lucy
18 : on
19 : the
20 : was
21 : armando
22 : become
23 : christian
24 : did
25 : famous
26 : perez
27 : mother
28 : old
29 : world
30 : youngest
31 : actor
32 : adult
33 : an
34 : film
35 : much
36 : paid
37 : debt
38 : limit
39 : national
40 : us
41 : athenians
42 : make
43 : money
44 : bird
45 : family
46 : owl
47 : what
48 : do
49 : from
50 : me
51 : quotes
52 : want
53 : you
54 : be
55 : get
56 : have
57 : olympics
58 : to
59 : cancer
60 : does
61 : one
62 : pancreatic
63 : israel
64 : judges
65 : many
66 : of
67 : floyd
68 : happened
69 : pink
70 : bombing
71 : city
72 : killed
73 : oklahoma
74 : people
75 : were
76 : book
77 : calc
78 : flatlands
79 : pre
80 : related
81 : 15
82 : 22
83 : a
84 : ah
85 : at
86 : battery
87 : can
88 : capacity
89 : charge
90 : electrons
91 : full
92 : hold
93 : v
94 : with
9

In [19]:
import pprint
bow_corpus =[dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
   print(sent)
   print(embedding)

question_orig ='How are epithelial tissues joined together ?'
question = clean_sentence(question_orig , stopwords = False);
question_embedding = dictionary.doc2bow(question.split())
print("\n\n",question,"\n",question_embedding)

Streaming output truncated to the last 5000 lines.
how long can run flats run before changing
[(2, 1), (15, 1), (87, 1), (430, 2), (750, 1), (1021, 1), (1022, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many acres is cooperstown dream park
[(2, 1), (10, 1), (65, 1), (160, 1), (825, 1), (1023, 1), (1024, 1)]
how many bass species are there
[(0, 1), (2, 1), (65, 1), (97, 1), 

In [20]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embedding,FAQdf,sentences):
  max_sim = -1;
  index_sim = -1;
  for index,faq_embedding in enumerate(sentence_embedding):
    sim = cosine_similarity(embedding,question_embedding)[0][0];
    print(index,sim,sentences[index])
    if sim > max_sim :
      max_sim = sim;
      index_sim = index;
  
  print("\n")
  print("Question:", question_orig)
  print("\n")
  print("Retrieved:",df.iloc[index_sim,0])
  print(df.iloc[index_sim,1])
retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

Streaming output truncated to the last 5000 lines.
6765 0.07669649888473704 what species is a spider
6766 0.07669649888473704 what species is a spider
6767 0.07669649888473704 what species is a spider
6768 0.07669649888473704 what species is a spider
6769 0.07669649888473704 what species is a spider
6770 0.07669649888473704 what species is a spider
6771 0.07669649888473704 what species is a spider
6772 0.07669649888473704 what species is a spider
6773 0.07669649888473704 what species is a spider
6774 0.07669649888473704 what species is a spider
6775 0.07669649888473704 what species is a spider
6776 0.07669649888473704 what species is a spider
6777 0.07669649888473704 what species is a spider
6778 0.07669649888473704 what species is a spider
6779 0.07669649888473704 what species is a spider
6780 0.07669649888473704 what species is a spider
6781 0.07669649888473704 what species is a spider
6782 0.07669649888473704 what species is a spider
6783 0.07669649888473704 what species is a spider

In [21]:
glove_model = None;
try:
  glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
  print("loaded glove model")

except:
  glove_model = api.load("glove-twitter-25")
  glove_model.save("./glovemodel.mod")
  print("saved glove model")
v2w_model = None;

[==================================================] 100.0% 104.8/104.8MB downloaded
saved glove model


In [ ]:
try:
  v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
  print('loaded w2v model')

except:
  v2w_model = api.load("word2vec-google-news-300")
  v2w_model.save("./w2vecmodel.mod")
  print("saved glove model")

w2vec_embedding_size = len(v2w_model["computer"]);
glove_embedding_size = len(glove_model["computer"]);

[==================================================] 100.0% 1662.8/1662.8MB downloaded
saved glove model


In [ ]:
def getWordVec(word,model):
  samp = model["computer"];
  vec = [0] * len(samp);
  try:
    vec = model[word];
  except:
    vec = [0] * len(samp);
  return(vec)

In [ ]:
def getPhaseEmbedding(phase,embedding_model):
  samp = getWordVec('computer',embedding_model);
  vec = [0] *len(samp);
  den = 0
  for word in phase.split():
    print(word)
    den=den+1;
    vec = vec +numpy.array(getWordVec(word,embedding_model));
  return vec.reshape(1,-1)

